In [26]:
import cv2
import glob
import numpy as np

In [27]:
vel_files= glob.glob('./test*/AutoEncoder/in*', recursive=True)

In [28]:
vel_files2=glob.glob('./test*/Conv*/in*', recursive=True)

In [29]:
img = cv2.imread(vel_files[8])

In [30]:
for tr in vel_files2:
    img2=cv2.imread(tr)
    
    if np.array_equal(img2, img):
        print(tr)
        print(vel_files[8])

./test_simple/ConvConvFC/in_54.png
./test_simple/AutoEncoder/in_74.png
